# HMM Regressor

Notebook applying HMM regressor to make predictions on player performance.

## Note

* CTRL-F and search for `TODO` to modify parts dependant of the nhl season.

## 1. Preparation
---

In [6]:
import pandas as pd
import numpy as np
import re
import math
import pickle
import glob
import xgboost
import tsfresh
import os
import glob
from scipy import stats
from hmmlearn import hmm
from datetime import datetime
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_context('notebook')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 100, 'linewidths':0}
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.mode.chained_assignment = None  # default='warn'
np.random.seed(42)

# Variables
scrape = False

### 1.1 Example

In [4]:
model = hmm.GaussianHMM(n_components=3, covariance_type="full")
model.startprob_ = np.array([0.6, 0.3, 0.1])
model.transmat_ = np.array([[0.7, 0.2, 0.1],
                            [0.3, 0.5, 0.2],
                            [0.3, 0.3, 0.4]])
model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
model.covars_ = np.tile(np.identity(2), (3, 1, 1))
X, Z = model.sample(100)

## 2. Get Predictions
---
### 2.1 Real Data

In [7]:
all_stats = {}
filenames = glob.glob(os.path.join('../LSTM-Neural-Network-for-Time-Series-Prediction/data', '*.csv'))
for f in filenames:
    print(f.split('/')[-1])
    if not os.path.basename(f) in all_stats.keys():
        data = pd.read_csv(f)
        data['Date'] = pd.to_datetime(data['Date'])
        # TODO: Change dates to be 2017-2020
        data = data[data['Date'] < pd.datetime(2020, 10, 1)]
        data = data[data['Date'] > pd.datetime(2017, 10, 1)]
        if data.shape[0] < 25:
            all_stats[f.split('/')[-1]] = 0
            continue
        X = np.array(data['Total_Points']).reshape(-1, 1)
        # X = np.array(pd.read_csv('../LSTM-Neural-Network-for-Time-Series-Prediction/data/NATHAN-MACKINNON-8477492.csv')['Total_Points']).reshape(-1, 1)
        if np.unique(X).shape == 1:
            all_stats[f.split('/')[-1]] = 0
            continue
        if X.shape[0] == 0:
            all_stats[f.split('/')[-1]] = 0
            continue
        n_states = int(X.max() + 1)
        model = hmm.GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=1000)
        model = model.fit(X)
        model.monitor_.converged

        points = np.array([])
        for i in np.arange(0,500):
            tmp = model.sample(82)[0].sum()
            points = np.append(points, tmp)

        statistics = stats.describe(points)
        all_stats[f.split('/')[-1]] = statistics

A.J.-GREER-8478421.csv
AARON-DELL-8477180.csv
AARON-EKBLAD-8477932.csv
AARON-NESS-8474604.csv
AARON-PALUSHAJ-8474030.csv
AARON-ROME-8470310.csv
AARON-VOLPATTI-8475619.csv
ADAM-ALMQUIST-8475332.csv
ADAM-BOQVIST-8480871.csv
ADAM-BROOKS-8478996.csv
ADAM-BURISH-8470063.csv
ADAM-CLENDENING-8476431.csv
ADAM-CRACKNELL-8471490.csv
ADAM-ERNE-8477454.csv
ADAM-FOX-8479323.csv
ADAM-GAUDETTE-8478874.csv
ADAM-HALL-8467925.csv
ADAM-HENRIQUE-8474641.csv
ADAM-JOHNSON-8480341.csv
ADAM-LARSSON-8476457.csv
ADAM-LOWRY-8476392.csv
ADAM-MCQUAID-8471717.csv
ADAM-PARDY-8471385.csv
ADAM-PAYERL-8476522.csv
ADAM-PELECH-8476917.csv
ADAM-WERNER-8479530.csv
ADAM-WILCOX-8476330.csv
ADIN-HILL-8478499.csv
ADRIAN-KEMPE-8477960.csv
AL-MONTOYA-8471219.csv
ALAN-QUINE-8476409.csv
ALEC-MARTINEZ-8474166.csv
ALEKSANDER-BARKOV-8477493.csv
ALEKSI-SAARELA-8478839.csv
ALES-HEMSKY-8469466.csv
ALEX-BIEGA-8473415.csv
ALEX-BROADHURST-8476283.csv
ALEX-BURROWS-8470358.csv
ALEX-CHIASSON-8475163.csv
ALEX-DEBRINCAT-8479337.csv
ALEX-FORMENT

Error: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.


ALEXANDER-YELESIN-8481630.csv
ALEXANDRE-BOLDUC-8470719.csv
ALEXANDRE-CARRIER-8478851.csv
ALEXANDRE-FORTIN-8479657.csv
ALEXANDRE-GRENIER-8476414.csv
ALEXANDRE-TEXIER-8480074.csv
ALEXEI-EMELIN-8471296.csv
ALEXEY-MARCHENKO-8476289.csv
ANDERS-BJORK-8478075.csv
ANDERS-LEE-8475314.csv
ANDERS-LINDBACK-8474765.csv
ANDERS-NILSSON-8475195.csv
ANDRE-BENOIT-8470378.csv
ANDRE-BURAKOVSKY-8477444.csv
ANDREAS-ATHANASIOU-8476960.csv
ANDREAS-BORGMAN-8480158.csv
ANDREAS-ENGLUND-8477971.csv
ANDREAS-JOHNSSON-8477341.csv
ANDREAS-MARTINSEN-8478561.csv
ANDREI-CHIBISOV-8481642.csv
ANDREI-LOKTIONOV-8474681.csv
ANDREI-MARKOV-8467496.csv
ANDREI-MIRONOV-8478842.csv
ANDREI-SVECHNIKOV-8480830.csv
ANDREI-VASILEVSKIY-8476883.csv
ANDREJ-MESZAROS-8471236.csv
ANDREJ-NESTRASIL-8475250.csv
ANDREJ-SEKERA-8471284.csv
ANDREJ-SUSTR-8477205.csv
ANDREW-AGOZZINO-8475461.csv
ANDREW-ALBERTS-8469626.csv
ANDREW-BODNARCHUK-8473550.csv
ANDREW-CAMPBELL-8474638.csv
ANDREW-COGLIANO-8471699.csv
ANDREW-COPP-8477429.csv
ANDREW-CRESCENZI-8476

Error: Number of distinct clusters (5) found smaller than n_clusters (6). Possibly due to duplicate points in X.


ANTHONY-DUCLAIR-8477407.csv
ANTHONY-GRECO-8479447.csv
ANTHONY-MANTHA-8477511.csv
ANTHONY-PELUSO-8474061.csv
ANTHONY-RICHARD-8478409.csv
ANTHONY-STOLARZ-8476932.csv
ANTOINE-BIBEAU-8477312.csv
ANTOINE-ROUSSEL-8474849.csv
ANTOINE-VERMETTE-8468535.csv
ANTON-BELOV-8477289.csv
ANTON-BLIDH-8477320.csv
ANTON-FORSBERG-8476341.csv
ANTON-KHUDOBIN-8471418.csv
ANTON-LANDER-8475165.csv
ANTON-LINDHOLM-8478073.csv
ANTON-RODIN-8475210.csv
ANTON-SLEPYSHEV-8477415.csv
ANTON-STRALMAN-8471873.csv
ANTON-VOLCHENKOV-8468501.csv
ANTON-WEDIN-8481637.csv
ANTTI-NIEMI-8474550.csv
ANTTI-RAANTA-8477293.csv
ANTTI-SUOMELA-8480965.csv
ANZE-KOPITAR-8471685.csv
ARRON-ASHAM-8465025.csv
ARTEM-ANISIMOV-8473573.csv
ARTEMI-PANARIN-8478550.csv
ARTTURI-LEHKONEN-8477476.csv
ASHTON-SAUTNER-8477085.csv
AUSTIN-CZARNIK-8478512.csv
AUSTIN-POGANSKI-8478040.csv
AUSTIN-WAGNER-8478455.csv
AUSTIN-WATSON-8475766.csv
AUSTON-MATTHEWS-8479318.csv
B.J.-CROMBEEN-8470647.csv
BARCLAY-GOODROW-8476624.csv
BARRET-JACKMAN-8467890.csv
BARRETT-HAYTON-8

...

SAMUEL-HENLEY-8476511.csv
SAMUEL-MORIN-8477502.csv
SCOTT-CLEMMENSEN-8466339.csv
SCOTT-DARLING-8474152.csv
SCOTT-FOSTER-8479138.csv
SCOTT-GOMEZ-8467351.csv
SCOTT-HANNAN-8466158.csv
SCOTT-HARRINGTON-8476449.csv
SCOTT-HARTNELL-8468486.csv
SCOTT-KOSMACHUK-8476910.csv
SCOTT-LAUGHTON-8476872.csv
SCOTT-MAYFIELD-8476429.csv
SCOTT-SABOURIN-8477149.csv
SCOTT-WEDGEWOOD-8475809.csv
SCOTT-WILSON-8476293.csv
SCOTTIE-UPSHALL-8470105.csv


Error: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.


SEAN-BERGENHEIM-8470176.csv
SEAN-COLLINS-8474744.csv
SEAN-COUTURIER-8476461.csv
SEAN-KURALY-8476374.csv
SEAN-MALONE-8477391.csv
SEAN-MONAHAN-8477497.csv
SEAN-WALKER-8480336.csv
SEBASTIAN-AHO-8478427.csv
SEBASTIAN-AHO-8480222.csv
SEMYON-VARLAMOV-8473575.csv
SERGEI-BOBROVSKY-8475683.csv
SERGEI-GONCHAR-8458951.csv
SERGEI-PLOTNIKOV-8478928.csv
SERGEY-KALININ-8478566.csv
SERGEY-TOLCHINSKY-8477566.csv
SETH-GRIFFITH-8476495.csv
SETH-HELGESON-8475274.csv
SETH-JONES-8477495.csv
SHANE-DOAN-8462038.csv
SHANE-GERSICH-8478063.csv
SHANE-HARPER-8475597.csv
SHANE-O'BRIEN-8470839.csv
SHANE-PRINCE-8476386.csv
SHAWN-HORCOFF-8467423.csv
SHAWN-MATTHIAS-8473574.csv
SHAWN-THORNTON-8465978.csv
SHAYNE-GOSTISBEHERE-8476906.csv
SHEA-THEODORE-8477447.csv
SHEA-WEBER-8470642.csv
SHELDON-BROOKBANK-8469992.csv
SHELDON-DRIES-8480326.csv
SHELDON-REMPAL-8480776.csv
SIDNEY-CROSBY-8471675.csv
SIMON-DESPRES-8475155.csv
SIMON-GAGNE-8467346.csv
SIMON-MOSER-8477666.csv
SLATER-KOEKKOEK-8476886.csv
SLAVA-VOYNOV-8474594.csv
SONN

Error: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.


STU-BICKEL-8474772.csv
STUART-PERCY-8476478.csv
SVEN-ANDRIGHETTO-8477413.csv
SVEN-BAERTSCHI-8476466.csv
TAGE-THOMPSON-8479420.csv
TANNER-FRITZ-8479206.csv
TANNER-GLASS-8470854.csv
TANNER-KERO-8478528.csv
TANNER-PEARSON-8476871.csv
TANNER-RICHARD-8476911.csv
TARO-HIROSE-8481433.csv
TAYLOR-BECK-8475192.csv
TAYLOR-CHORNEY-8471710.csv
TAYLOR-FEDUN-8476166.csv
TAYLOR-HALL-8475791.csv
TAYLOR-LEIER-8476955.csv
TAYLOR-PYATT-8467881.csv
TEDDY-BLUEGER-8476927.csv
TEDDY-PURCELL-8473962.csv
TEEMU-PULKKINEN-8475800.csv
TEEMU-SELANNE-8457981.csv
TEUVO-TERAVAINEN-8476882.csv
THATCHER-DEMKO-8477967.csv
THOMAS-CHABOT-8478469.csv
THOMAS-DI-PAULI-8476974.csv
THOMAS-GREISS-8471306.csv
THOMAS-HICKEY-8474066.csv
THOMAS-VANEK-8470598.csv
TIM-ERIXON-8475148.csv
TIM-GLEASON-8469476.csv
TIM-HEED-8475841.csv
TIM-JACKMAN-8469490.csv
TIM-KENNEDY-8471842.csv
TIM-SCHALLER-8477213.csv
TIM-SESTITO-8471636.csv
TIM-THOMAS-8460703.csv
TIMO-MEIER-8478414.csv
TIMOTHY-GETTINGER-8479364.csv
TIMOTHY-LILJEGREN-8480043.csv
TJ-B

### 2.2 Save Results

In [8]:
all_stats_df = pd.DataFrame.from_dict(all_stats, orient='columns').T
all_stats_df.columns = ['nobs', 'minmax', 'mean', 'variance', 'skewness', 'kurtosis']
all_stats_df.to_pickle('predictions_hmm_3years_for_2021.pkl')
all_stats_df.head(10)

,nobs,minmax,mean,variance,skewness,kurtosis
A.J.-GREER-8478421.csv,500,"(5.709672724683751, 25.534055004993228)",13.1009,7.60412,0.227929,0.522997
AARON-DELL-8477180.csv,500,"(-0.23544148291288167, 4.213466197174426)",0.925224,0.929644,0.854183,0.174074
AARON-EKBLAD-8477932.csv,500,"(18.98625378121845, 58.96552562304788)",40.52,48.4988,0.0443088,-0.160787
AARON-NESS-8474604.csv,500,"(-0.0400711438583026, 12.910477808168023)",5.11348,4.6331,0.482452,0.312154
AARON-PALUSHAJ-8474030.csv,0,0,0,0,0,0
AARON-ROME-8470310.csv,0,0,0,0,0,0
AARON-VOLPATTI-8475619.csv,0,0,0,0,0,0
ADAM-ALMQUIST-8475332.csv,0,0,0,0,0,0
ADAM-BOQVIST-8480871.csv,500,"(10.085345063158206, 54.673079579186656)",26.9481,58.5617,0.53842,0.198286
ADAM-BROOKS-8478996.csv,0,0,0,0,0,0
